# Konjunkturprognose Tracker
November 2025

## 0) Setup


In [322]:
import pandas as pd
from playwright.async_api import async_playwright
import time
import aiohttp
import asyncio
import requests
import io
from io import StringIO
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
import os
import json 
import pygsheets
from datawrapper import Datawrapper

## 1) Get Data

### Konjunkturprognosen

In [323]:
# Daten von Google Sheets laden
SHEET_ID = "1_GuhaEY2LHJeSgae8tiioBCTsnQ3YzrC3BbVCybY4A0" 
gc = pygsheets.authorize(service_file='/Users/bb/Desktop/handelsblatt/Konjunkturprognosetracker/credentials.json')
sh = gc.open_by_key(SHEET_ID)
ws1 = sh[3] 
df_prog = pd.DataFrame(ws1.get_all_records())

# Spalten auswählen und sortieren; Numerische Umwandlung
df_prog = df_prog[["Year", "Value", "Institute", "Release Season", "Horizon", "Vintage"]]
df_prog["Year"] = pd.to_numeric(df_prog["Year"], errors="coerce")
df_prog = df_prog.sort_values("Year").reset_index(drop=True)
df_prog

,Year,Value,Institute,Release Season,Horizon,Vintage
0,2000,3.0,Kiel Institute,Autumn,0,2000-09-19
1,2000,2.7,ifo,Summer,0,2000-07-27
2,2000,2.4,DIW,Winter,-1,2000-01-05
3,2000,2.8,IWH,Winter,1,1999-12-20
4,2000,2.5,IWH,Summer,1,1999-07-02
...,...,...,...,...,...,...
990,2027,0.6,IWH,Autumn,2,2025-09-04
991,2027,1.0,IWH,Winter,2,2025-12-11
992,2027,1.8,DIW,Autumn,2,2025-09-04
993,2027,1.6,ifo,Autumn,2,2025-09-04


### Eingetroffene Konjunkturdaten

In [324]:
# Daten laden
data = sh[4] 
df_real = pd.DataFrame(data).iloc[27:45, [0, 7]]

df_real = df_real.rename(columns={
    df_real.columns[0]: 'Year',
    df_real.columns[1]: 'Value'  
})

# Numerische Umwandlung
df_real['Year'] = pd.to_numeric(df_real['Year'], errors='coerce')
df_real['Value'] = df_real['Value'].str.replace(',', '.', regex=False)
df_real['Value'] = df_real['Value'].str.replace('– ', '-', regex=False)  
df_real['Value'] = pd.to_numeric(df_real['Value'], errors='coerce')
df_real


,Year,Value
27,2005,0.7
28,2006,3.8
29,2007,3.0
30,2008,1.0
31,2009,-5.7
32,2010,4.2
33,2011,3.9
34,2012,0.4
35,2013,0.4
36,2014,2.2


## 2) Transform Data

### Medianabweichung berechnen 

In [325]:
# Daten zusammenführen
df = df_prog.merge(df_real[['Year', 'Value']], on='Year', suffixes=('_prog', '_real'))

# Abweichung berechnen
df['diff'] = df['Value_prog'] - df['Value_real']

# in positive und negative Abweichungen unterteilen
df['higher'] = (df['diff'] > 0).astype(int)

# Median Abweichung pro Institut, Prognosezeitraum und positive/negative Abweichung
df['median_diff'] = df.groupby(['Institute', 'Horizon', 'higher'])['diff'].transform('median')

# Nur zur Überprüfung: Anzahl der Beobachtungen pro Median-Berechnung
df['count'] = df.groupby(['Institute', 'Horizon', 'higher'])['diff'].transform('count')

df_constant = df.copy()


### Abweichungen kompakt darstellen

In [326]:
# Nur die positive/negative Medianabweichung  pro Institut und Prognosezeitpunkt
df_wide = (
    df
    .loc[:, ["Institute", "Horizon", "higher", "median_diff"]]
    .drop_duplicates()
    .pivot(
        index=["Institute", "Horizon"],
        columns="higher",
        values="median_diff"
    )
    .reset_index()
    .rename(columns={
        0: "median_diff_lower",
        1: "median_diff_higher"
    })
)



### Datenset vorbereiten

In [327]:
# Sicherstellen, dass Daten im richtigen Format sind; Datenset sortieren
df_prog['Year'] = pd.to_numeric(df_prog['Year'], errors='coerce')
df_prog['Vintage'] = pd.to_datetime(df_prog['Vintage'], errors='coerce')
df_sorted = df_prog.sort_values(['Year', 'Institute', 'Vintage'], ascending=[True, True, False])

# Nur die neueste Prognose pro Jahr und Institut behalten 
df_sorted = df_sorted.groupby(['Year', 'Institute'], as_index=False).first()

# Datenset einzelnd pro Jahr trennen
df_2025 = df_sorted[df_sorted['Year'] == 2025].copy()
df_2026 = df_sorted[df_sorted['Year'] == 2026].copy()
df_2027 = df_sorted[df_sorted['Year'] == 2027].copy()  

df

,Year,Value_prog,Institute,Release Season,Horizon,Vintage,Value_real,diff,higher,median_diff,count
0,2005,0.9,RWI,Autumn,0,2005-09-19,0.7,0.2,1,0.55,24
1,2005,1.3,Kiel Institute,Summer,1,2004-06-15,0.7,0.6,1,1.20,41
2,2005,1.0,RWI,Spring,0,2005-03-02,0.7,0.3,1,0.55,24
3,2005,1.8,RWI,Summer,1,2004-07-28,0.7,1.1,1,1.30,31
4,2005,1.3,IWH,Winter,1,2004-12-20,0.7,0.6,1,1.10,37
...,...,...,...,...,...,...,...,...,...,...,...
680,2022,4.9,DIW,Autumn,1,2021-09-16,1.8,3.1,1,1.30,32
681,2022,4.3,ifo,Summer,1,2021-06-01,1.8,2.5,1,1.50,27
682,2022,3.9,RWI,Winter,1,2021-12-15,1.8,2.1,1,1.30,31
683,2022,3.6,IWH,Spring,1,2021-03-11,1.8,1.8,1,1.10,37


In [328]:
dfs = {
    2025: df_2025.copy(),
    2026: df_2026.copy(),
    2027: df_2027.copy()
}

for year, df in dfs.items():   
    dfs[year] = df.merge(
        df_wide,
        on=["Institute", "Horizon"],
        how="left"
    )

df_2025 = dfs[2025]
df_2026 = dfs[2026]
df_2027 = dfs[2027]

dfs[2027]


,Year,Institute,Value,Release Season,Horizon,Vintage,median_diff_lower,median_diff_higher
0,2027,DIW,1.6,Winter,2,2025-12-12,-1.50,0.80
1,2027,IWH,1.0,Winter,2,2025-12-11,-1.80,1.10
2,2027,Kiel Institute,1.3,Winter,2,2025-12-11,-1.75,1.10
3,2027,RWI,1.4,Winter,2,2025-12-11,-1.70,0.85
4,2027,ifo,1.1,Winter,2,2025-12-11,-1.75,1.30


In [329]:
final_dfs = {}

for year, df in dfs.items():  # unpack (key, value)
    df_copy = df.copy()  # sichere Kopie

    df_final = df_copy[
        ["Year", "Institute", "Value", "median_diff_lower", "median_diff_higher"]
    ].copy()

    # Berechne Lower/Higher
    df_final["Lower"] = df_final["Value"] + df_final["median_diff_lower"]
    df_final["Higher"] = df_final["Value"] + df_final["median_diff_higher"]

    # Endgültige Spalten
    df_final = df_final[["Year", "Institute", "Value", "Lower", "Higher"]]

    final_dfs[year] = df_final

# Ergebnisse wieder den Jahresvariablen zuweisen
df_2025_final = final_dfs[2025][['Institute', 'Lower', 'Value', 'Higher']].copy()
df_2026_final = final_dfs[2026][['Institute', 'Lower', 'Value', 'Higher']].copy()
df_2027_final = final_dfs[2027][['Institute', 'Lower', 'Value', 'Higher']].copy()



### Berechnung des jährlichen Medians (besser Durchschnitt?)

In [330]:
dfs = {
    2025: df_2025_final,
    2026: df_2026_final,
    2027: df_2027_final
}

for year, df in dfs.items():
    df_copy = df.copy()
    avg_row = {
        "Year": year,
        "Institute": "Average",
        "Value": df_copy["Value"].median(),
        "Lower": df_copy["Lower"].median(),
        "Higher": df_copy["Higher"].median()
    }

    dfs[year] = pd.concat(
        [df_copy, pd.DataFrame([avg_row])],
        ignore_index=True
    )

df_2025_final = dfs[2025][['Institute', 'Lower', 'Value', 'Higher']].round(2).copy()
df_2026_final = dfs[2026][['Institute', 'Lower', 'Value', 'Higher']].round(2).copy()
df_2027_final = dfs[2027][['Institute', 'Lower', 'Value', 'Higher']].copy()

df_2025_final


,Institute,Lower,Value,Higher
0,DIW,-0.5,0.2,1.00
1,IWH,-0.5,0.2,0.70
2,Kiel Institute,-0.5,0.1,0.55
3,RWI,-0.6,0.1,0.65
4,ifo,-0.5,0.1,0.60
5,Average,-0.5,0.1,0.65


alle Jahre

In [331]:
df_constant2 = df_constant.copy()
df_constant2


,Year,Value_prog,Institute,Release Season,Horizon,Vintage,Value_real,diff,higher,median_diff,count
0,2005,0.9,RWI,Autumn,0,2005-09-19,0.7,0.2,1,0.55,24
1,2005,1.3,Kiel Institute,Summer,1,2004-06-15,0.7,0.6,1,1.20,41
2,2005,1.0,RWI,Spring,0,2005-03-02,0.7,0.3,1,0.55,24
3,2005,1.8,RWI,Summer,1,2004-07-28,0.7,1.1,1,1.30,31
4,2005,1.3,IWH,Winter,1,2004-12-20,0.7,0.6,1,1.10,37
...,...,...,...,...,...,...,...,...,...,...,...
680,2022,4.9,DIW,Autumn,1,2021-09-16,1.8,3.1,1,1.30,32
681,2022,4.3,ifo,Summer,1,2021-06-01,1.8,2.5,1,1.50,27
682,2022,3.9,RWI,Winter,1,2021-12-15,1.8,2.1,1,1.30,31
683,2022,3.6,IWH,Spring,1,2021-03-11,1.8,1.8,1,1.10,37


In [332]:

df_h0 = df_constant2[df_constant2['higher'] == 0][["Horizon", "Institute", "median_diff"]].copy()

In [333]:


df_h0 = df_h0.rename(columns={"median_diff": "median_diff_negativ"})

df_constant2 = df_constant2.merge(
    df_h0,
    on=["Horizon", "Institute"],
    how="left"
)


In [334]:
df_constant2

,Year,Value_prog,Institute,Release Season,Horizon,Vintage,Value_real,diff,higher,median_diff,count,median_diff_negativ
0,2005,0.9,RWI,Autumn,0,2005-09-19,0.7,0.2,1,0.55,24,-0.70
1,2005,0.9,RWI,Autumn,0,2005-09-19,0.7,0.2,1,0.55,24,-0.70
2,2005,0.9,RWI,Autumn,0,2005-09-19,0.7,0.2,1,0.55,24,-0.70
3,2005,0.9,RWI,Autumn,0,2005-09-19,0.7,0.2,1,0.55,24,-0.70
4,2005,0.9,RWI,Autumn,0,2005-09-19,0.7,0.2,1,0.55,24,-0.70
...,...,...,...,...,...,...,...,...,...,...,...,...
18923,2022,3.6,IWH,Spring,1,2021-03-11,1.8,1.8,1,1.10,37,-1.45
18924,2022,3.0,DIW,Winter,-1,2022-02-23,1.8,1.2,1,1.10,6,-1.90
18925,2022,3.0,DIW,Winter,-1,2022-02-23,1.8,1.2,1,1.10,6,-1.90
18926,2022,3.0,DIW,Winter,-1,2022-02-23,1.8,1.2,1,1.10,6,-1.90


In [335]:
df_h1 = df_constant2[df_constant2["higher"] == 1][["Horizon", "Institute", "median_diff"]].copy()
df_h1 = df_h1.rename(columns={"median_diff": "median_diff_positiv"})

df_constant2 = df_constant2.merge(
    df_h1,
    on=["Horizon", "Institute"],
    how="left"
)


In [336]:
df_constant2 = df_constant2.drop_duplicates()
df_constant2

,Year,Value_prog,Institute,Release Season,Horizon,Vintage,Value_real,diff,higher,median_diff,count,median_diff_negativ,median_diff_positiv
0,2005,0.9,RWI,Autumn,0,2005-09-19,0.7,0.2,1,0.55,24,-0.70,0.55
26136,2005,1.3,Kiel Institute,Summer,1,2004-06-15,0.7,0.6,1,1.20,41,-1.00,1.20
65537,2005,1.0,RWI,Spring,0,2005-03-02,0.7,0.3,1,0.55,24,-0.70,0.55
91673,2005,1.8,RWI,Summer,1,2004-07-28,0.7,1.1,1,1.30,31,-1.10,1.30
106677,2005,1.3,IWH,Winter,1,2004-12-20,0.7,0.6,1,1.10,37,-1.45,1.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17772789,2022,4.9,DIW,Autumn,1,2021-09-16,1.8,3.1,1,1.30,32,-1.20,1.30
17786901,2022,4.3,ifo,Summer,1,2021-06-01,1.8,2.5,1,1.50,27,-1.10,1.50
17794704,2022,3.9,RWI,Winter,1,2021-12-15,1.8,2.1,1,1.30,31,-1.10,1.30
17809708,2022,3.6,IWH,Spring,1,2021-03-11,1.8,1.8,1,1.10,37,-1.45,1.10


In [337]:
# Upper / Lower berechnen
df_constant2["Upper"] = df_constant2["Value_prog"] + df_constant2["median_diff_positiv"]
df_constant2["Lower"] = df_constant2["Value_prog"] + df_constant2["median_diff_negativ"]

# Optional: nur relevante Spalten behalten
df_result = df_constant2[["Year", "Institute", "Value_prog", "Value_real", "Upper", "Lower", "Horizon", "higher", "Vintage"]]

df_result


,Year,Institute,Value_prog,Value_real,Upper,Lower,Horizon,higher,Vintage
0,2005,RWI,0.9,0.7,1.45,0.20,0,1,2005-09-19
26136,2005,Kiel Institute,1.3,0.7,2.50,0.30,1,1,2004-06-15
65537,2005,RWI,1.0,0.7,1.55,0.30,0,1,2005-03-02
91673,2005,RWI,1.8,0.7,3.10,0.70,1,1,2004-07-28
106677,2005,IWH,1.3,0.7,2.40,-0.15,1,1,2004-12-20
...,...,...,...,...,...,...,...,...,...
17772789,2022,DIW,4.9,1.8,6.20,3.70,1,1,2021-09-16
17786901,2022,ifo,4.3,1.8,5.80,3.20,1,1,2021-06-01
17794704,2022,RWI,3.9,1.8,5.20,2.80,1,1,2021-12-15
17809708,2022,IWH,3.6,1.8,4.70,2.15,1,1,2021-03-11


In [338]:
df_result = df_result.drop_duplicates(subset=["Year", "Institute", "Horizon"], keep="first").reset_index(drop=True)


In [345]:
df_result

,Year,Institute,Value_prog,Value_real,Upper,Lower,Horizon,higher,Vintage
0,2005,RWI,0.9,0.7,1.45,0.20,0,1,2005-09-19
1,2005,Kiel Institute,1.3,0.7,2.50,0.30,1,1,2004-06-15
2,2005,RWI,1.8,0.7,3.10,0.70,1,1,2004-07-28
3,2005,IWH,1.3,0.7,2.40,-0.15,1,1,2004-12-20
4,2005,Kiel Institute,0.7,0.7,1.15,0.10,0,0,2005-09-07
...,...,...,...,...,...,...,...,...,...
244,2022,RWI,1.9,1.8,2.45,1.20,0,1,2022-06-15
245,2022,DIW,4.3,1.8,5.60,3.10,1,1,2021-06-17
246,2022,ifo,3.2,1.8,4.70,2.10,1,1,2021-03-24
247,2022,IWH,3.5,1.8,4.60,2.05,1,1,2021-12-14


In [347]:
df_wide = df_result.pivot_table(
    index='Year', 
    columns='Institute', 
    values='Value_prog', 
    aggfunc='mean'  # oder 'mean', 'max', je nach gewünschter Logik
)

value_real = df_result.groupby('Year')['Value_real'].first()
df_wide.insert(0, 'Value_real', value_real)

df_wide.columns.name = None
df_wide = df_wide.reset_index()

print(df_wide.head())


   Year  Value_real       DIW       IWH  Kiel Institute   RWI       ifo
0  2005         0.7  1.600000  1.333333        1.000000  1.35  1.250000
1  2006         3.8  1.666667  1.633333        1.733333  1.80  1.750000
2  2007         3.0  1.900000  1.700000        1.633333  2.00  2.250000
3  2008         1.0  2.233333  2.033333        1.933333  2.15  2.166667
4  2009        -5.7 -1.200000 -0.333333       -1.066667 -2.15 -2.333333


## 3) Visualize

In [ ]:
wk2025 = sh[0] 
wk2026 = sh[1] 
wk2027 = sh[2] 
wkALL = sh[5]
wk2025.clear()
wk2026.clear()
wk2026.clear()
wkALL.clear()

wk2025.set_dataframe(df_2025_final, (1,1)) 
wk2026.set_dataframe(df_2026_final, (1,1))
wk2027.set_dataframe(df_2027_final, (1,1))
wkALL.set_dataframe(df_wide, (1,1))

In [341]:
from datawrapper import Datawrapper

CREDENTIALS_PATH = "/Users/bb/Desktop/handelsblatt/Konjunkturprognosetracker/credentials_dw.json"

with open(CREDENTIALS_PATH, "r") as f:
    data = json.load(f)
    DWKEY = data["key"]


2025

In [342]:

dw = Datawrapper(access_token=DWKEY)
allcharts=dw.get_charts(limit=500)
filtered_charts=list(filter(lambda item: item["folderId"] == "onkX1", allcharts["list"]))
for item in (filtered_charts):
  print(item["id"])
  dw.publish_chart(chart_id=item["id"])

2026

In [343]:

dw = Datawrapper(access_token=DWKEY)
allcharts=dw.get_charts(limit=500)
filtered_charts=list(filter(lambda item: item["folderId"] == "s4Vk3", allcharts["list"]))
for item in (filtered_charts):
  print(item["id"])
  dw.publish_chart(chart_id=item["id"])

2027

In [344]:

dw = Datawrapper(access_token=DWKEY)
allcharts=dw.get_charts(limit=500)
filtered_charts=list(filter(lambda item: item["folderId"] == "3Yzvz", allcharts["list"]))
for item in (filtered_charts):
  print(item["id"])
  dw.publish_chart(chart_id=item["id"])

Über die Zeit